In [1]:
import json
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, GRU, LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D, MaxPooling1D
from keras.utils import np_utils
import itertools
import ast
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import sys, os
import traceback
from itertools import chain
import numpy as np
import pandas as pd


import tensorflow as tf
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

Using TensorFlow backend.


In [3]:
def read_data(rootdir):
    data_list = []
    for subdir, dirs, files in os.walk(rootdir):
        for fname in files:
            #print os.path.join(subdir, file)
            filepath = subdir + os.sep + fname
            print(filepath)
            with open(filepath) as f:
                all_lines = f.readlines()
                print(len(all_lines))
                for line1,line2 in zip(all_lines[::2], all_lines[1::2]):
#                     try:
                    text = ast.literal_eval(line1)
                    label = ast.literal_eval(line2)

#                         for i in range(len(label)):
#                             if label[i] == 'NA':
#                                 label[i] = 'O'
                    n_labels = set(label)
                    if len(text) > 3 and len(n_labels) > 1:
                        sentence = [(a,b) for a,b in zip(text, label)]
                        sentence.append(('.', 'O'))
                        sentence = [item for item in sentence if item[0].strip() != '']
                        data_list.append(sentence)
#                     except Exception:
#                         pass
            f.close()
    print(len(data_list))
    return data_list

def get_sentences(data_list):
    sentences = []
    for a in data_list.article.unique():
        words = []
        prev_n = 0
        for _,s in data_list.loc[data_list.article == a].iterrows():
            n = s['sentence_no']
            if n != prev_n:
                if len(words) > 2 and len(set([x[1] for x in words])) > 1:
                    sentences.append(words)
                words = []
            words.append((s['word'], s['encoding'], s['part_of_speech_x']))
            prev_n = n
            
        if len(words) > 2 and len(set([x[1] for x in words])) > 1:
            sentences.append(words)
            
    return sentences

In [81]:
lucas = read_data('../src/Go_Deep/data/')
me = pd.read_csv('../data/go_cc_2.0.csv')
me.word.fillna('NA', inplace=True)

temp = pd.read_csv('../data/go_bpmf_2.0.csv')
temp.word.fillna('NA', inplace=True)
me = me.merge(temp, on=['article', 'sentence_no', 'word_no'])
me = me.rename(columns={'word_x' : 'word'})
x_counts = me.encoding_x.value_counts()
y_counts = me.encoding_y.value_counts()
me['encoding_x'] = me.encoding_x.apply(lambda x: x if x_counts[x] > 1 else 'O')
me['encoding_y'] = me.encoding_y.apply(lambda x: x if y_counts[x] > 1 else 'O')
me['encoding'] = [i if i != 'O' else j for i,j in zip(me.encoding_x, me.encoding_y)]
me = get_sentences(me)

../src/Go_Deep/data//merged_file.txt
38114
9571


In [74]:
print(len(lucas), len(me))

9571 10982


In [75]:
lucas_df = pd.DataFrame([(w[0],w[1],i) for s, i in zip(lucas, range(len(lucas))) for w in s])
me_df = pd.DataFrame([(w[0],w[1],i) for s, i in zip(me, range(len(me))) for w in s])

In [76]:
lucas_enc = lucas_df[lucas_df[1] != 'O'][1].unique()

In [77]:
me_enc = me_df[me_df[1] != 'O'][1].unique()

In [78]:
len([i for i in lucas_enc if i not in me_enc])

53

In [79]:
len([i for i in me_enc if i not in lucas_enc])

150

In [82]:
me = pd.read_csv('../data/go_cc_2.0.csv')
me.word.fillna('NA', inplace=True)

In [87]:
me[(me.article == '11597317.txt') & (me.encoding != 'O')]

,article,sentence_no,word,word_no,part_of_speech,encoding
7770,11597317.txt,10,Cells,0,NNS,GO:0005623
7780,11597317.txt,10,cells,10,NNS,GO:0005623
7787,11597317.txt,10,chromosomal,17,NN,GO:0005694
7897,11597317.txt,14,nuclear,5,JJ,GO:0005634
7931,11597317.txt,16,Cell,0,NNP,GO:0005623
7955,11597317.txt,16,cell,24,NN,GO:0005623
8017,11597317.txt,20,Cells,0,NNS,GO:0005623
8042,11597317.txt,20,complexes,25,NNS,GO:0032991
8069,11597317.txt,21,cells,16,NNS,GO:0005623
8109,11597317.txt,22,cells,29,NNS,GO:0005623


In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [10]:
a = device_lib.list_local_devices()

In [9]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [11]:
a

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2390883140899905363, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5186299574496497102
 physical_device_desc: "device: XLA_CPU device"]

In [12]:
os.environ["CUDA_DEVICE_ORDER"]

'PCI_BUS_ID'